# Characterize the PSF from sources Light Curve for LSSTComCamSim

- author Sylvie Dagoret-Campagne
- creation date 2024-06-04
- last update 2024-06-04
- affiliation : IJCLab
- Kernel **w_2024_16**

- tutorial : https://github.com/rubin-dp0/tutorial-notebooks/blob/main/DP02_12a_PSF_Data_Products.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils
import lsst.daf.base as dafBase

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom
from lsst.geom import Point2D, radToDeg, SpherePoint, degrees

In [ ]:
# Pipeline tasks
#from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
#from lsst.meas.algorithms.detection import SourceDetectionTask
#from lsst.meas.deblender import SourceDeblendTask
#from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import skew

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collectionn = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")

instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"
#3533 : No matching visitId
#tract = 2494
#tract = 7445
tract = 9880
band = "g"

suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
SIGMA_TO_FWHM = 2.0*np.sqrt(2.0*np.log(2.0))

**Function**: `getPsfProperties`

Given a PSF model (`lsst.meas.extensions.psfex.PsfexPsf`) and a coordinate where the model is being evaluated
(`lsst.geom.Point2D`) this functions returns the PSF FWHM, flux from aperture photometry, 
peak value of the normalized PSF (the PSF is normalized to a sum of 1), and size of the PSF postage stamp.

In [ ]:
def getPsfProperties(psf, point):
    """Function to obtain PSF properties.

    Parameters
    ----------
    psf : `lsst.meas.extensions.psfex.PsfexPsf`
        PSF object.
    point : `lsst.geom.Point2D`
        Coordinate where the PSF is being evaluated.

    Returns
    -------
    fwhm : `float`
        Full-width at half maximum: PSF determinant radius
        from SDSS adaptive moments matrix (sigma) times
        SIGMA_TO_FWHM.
    ap_flux : `float`
        PSF flux from aperture photometry weighted
        by a sinc function.
    peak : `float`
        Peak PSF value.
    dims : `lsst.geom.ExtendI`
        PSF postage stamp dimensions.
    """
    sigma = psf.computeShape(point).getDeterminantRadius()
    fwhm = sigma * SIGMA_TO_FWHM
    ap_flux = psf.computeApertureFlux(radius=sigma, position=point)
    peak = psf.computePeak(position=point)
    dims = psf.computeImage(point).getDimensions()

    print(f"PSF FWHM: {fwhm:.4} pix \n"
          f"PSF flux from aperture photometry: {ap_flux:.4} \n"
          f"Peak PSF value: {peak:.4} \n"
          f"PSF postage stamp dimensions: {dims} \n")

    return (sigma, ap_flux, peak, dims)

**Function**: `plotRadialAverage`

Given an image of the PSF (`lsst.afw.image.ExposureF`), this function 
plots the azimuthally-averaged radial profile of the PSF, and fits a Gaussian function to it.

In [ ]:
def plotRadialAverage(exp, ax=None):
    """
    Function to plot the radial average of a point spread function (PSF) image.

    Parameters
    ----------
    exp : `lsst.afw.image.ExposureF`
        The PSF image exposure.
    ax : `matplotlib.axes.Axes`, optional
        If provided, the plot will be drawn on this axis (default is None).

    Returns
    -------
    ax : `matplotlib.axes.Axes`
        The axis on which the plot was drawn.
    """
    data = exp.array
    xlen, ylen = data.shape
    center = np.array([xlen / 2, ylen / 2])
    distances = []
    values = []

    for i in range(xlen):
        for j in range(ylen):
            value = data[i, j]
            dist = np.linalg.norm(np.array([i, j]) - center)
            if dist <= xlen / 2:
                values.append(value)
                distances.append(dist)

    peakPos = 0
    amplitude = np.max(values)
    width = 10

    bounds = ((0, 0, 0), (np.inf, np.inf, np.inf))

    try:
        pars, pCov = curve_fit(gauss, distances, values,
                               [amplitude, peakPos, width], bounds=bounds)
        pars[0] = np.abs(pars[0])
        pars[2] = np.abs(pars[2])
    except RuntimeError:
        pars = None

    if ax is None:
        ax = plt.gca()

    ax.plot(distances, values, 'x', label='Radial average')

    if pars is not None:
        fitAmp = pars[0]
        fitGausMean = pars[1]
        fitFwhm = pars[2] * SIGMA_TO_FWHM
        fitline = gauss(distances, *pars)

        x_fit = np.linspace(-np.max(distances), np.max(distances), 100)
        y_fit = gauss(x_fit, *pars)
        skewness = skew(y_fit, axis=0, bias=True)

        ax.plot(distances, fitline,"r-",
                label=f" Gaussian Fit \n Amp: {fitAmp:.3f}"
                      f"\n Position: {fitGausMean:.2f}"
                      f"\n FWHM: {fitFwhm:.2f}"
                      f"\n \n Skewness: {skewness:.2f}")

    ax.set_ylabel('Flux (ADU)')
    ax.set_xlabel('Radius (pix)')
    ax.set_aspect(1.0 / ax.get_data_ratio(), adjustable='box')
    ax.legend()
    ax.set_title("Azimuthally-averaged radial profile")

    return ax

**Function**: `plotCurveOfGrowth`

Given an image of the PSF (`lsst.afw.image.ExposureF`), this function
plots the encircled energy of the PSF as a function of radius (curve of growth).

In [ ]:
def plotCurveOfGrowth(exp, ax=None):
    """
    Function to plot the curve of growth of a point
    spread function (PSF) image.

    Parameters
    ----------
    exp : `lsst.afw.image.ExposureF`
        The PSF image exposure.
    ax : `matplotlib.axes.Axes`, optional
        If provided, the plot will be drawn on this axis (default is None).

    Returns
    -------
    ax : `matplotlib.axes.Axes`
        The axis on which the plot was drawn.
    """
    data = exp.array
    xlen, ylen = data.shape
    center = np.array([xlen / 2, ylen / 2])
    distances = []
    values = []

    for i in range(xlen):
        for j in range(ylen):
            value = data[i, j]
            dist = np.linalg.norm(np.array([i, j]) - center)
            if dist <= xlen / 2:
                values.append(value)
                distances.append(dist)

    d = np.array([(r, v) for (r, v) in sorted(zip(distances, values))])
    radii = d[:, 0]
    values = d[:, 1]
    cum_fluxes = np.cumsum(values)
    cum_fluxes_norm = cum_fluxes / np.max(cum_fluxes)

    if ax is None:
        ax = plt.gca()

    ax.plot(radii, cum_fluxes_norm, markersize=10,color="b")
    ax.set_ylabel('Encircled flux (%)')
    ax.set_xlabel('Radius (pix)')
    ax.set_title("Encircled flux")

    return ax

**Function**: `plotContours` 

Given an image of the PSF (`lsst.afw.image.ExposureF`) and, optionally,
the plot axis and number of contours (`nContours`), this function
plots the two-dimensional contous of the PSF postage stamp.

In [ ]:
def plotContours(exp, ax=None, nContours=10):
    """
    Function to plot contour lines of a point spread function (PSF) image.

    Parameters
    ----------
    exp : `lsst.afw.image.ExposureF`
        The PSF image exposure.
    ax : `matplotlib.axes.Axes`, optional
        If provided, the plot will be drawn on this axis (default is None).
    nContours : int, optional
        The number of contour lines to plot (default is 10).

    Returns
    -------
    ax : `matplotlib.axes.Axes`
        The axis on which the plot was drawn.
    """
    data = exp.array
    xlen, ylen = data.shape

    vmin = np.percentile(data, 0.1)
    vmax = np.percentile(data, 99.9)
    lvls = np.linspace(vmin, vmax, nContours)

    xx, yy = np.meshgrid(np.linspace(-xlen / 2, xlen / 2, xlen),
                         np.linspace(-ylen / 2, ylen / 2, ylen))

    if ax is None:
        ax = plt.gca()

    #ax.contour(xx, yy, data, levels=lvls)
    ax.contour(xx, yy, np.flipud(data), levels=lvls)
    ax.tick_params(which="both", direction="in", top=True,
                   right=True, labelsize=8)
    ax.set_aspect("equal")
    ax.set_xlabel('x (pix)')
    ax.set_ylabel('y (pix)')
    ax.set_title("Contour plot")
    ax.set_xlim([-5, 5])
    ax.set_ylim([-5, 5])

    return ax

In [ ]:
def gauss(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
inputfilename = "sources_objectTable-t9880-bg-o2736-LSSTComCamSim_runs_nightlyvalidation_20240403_d_2024_03_29_DM-43612.csv"
tract = 9880
band = "g"
objectname = 2736
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
path = f"calexp_t{tract}_b{band}_o{objectname}"
title = f"LSSTComCamSim Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMJD",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMJD",y="psfMagDiffMmag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
bundle_selected_object = df_myselectedvisits[["visit","psfMagDiffMmag"]]
fig, ax = plt.subplots(1,1,figsize=(16,2))
bundle_selected_object.plot.bar(x="visit",ax=ax,color="b" ,rot=90,grid=True)
title = f"psfMagDiffMmag for object {objectname} for tract {tract} in band {band} (LSSTComCamSim)"
ax.set_title(title)

In [ ]:
print(path)

In [ ]:
listOfVisitId = df_myselectedvisits["visit"].values
listOfVisitId

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
datasetType = 'calexp'
boxSize = 25
imMin, imMax, Q = -0.001, 0.004, 8
expMin, expMax = -25, 100


all_xytarget = []
all_cutout = []
all_firstpsfimage = []
all_secondpsfimage = []
all_psfSigma = []
all_psfMagDiffMmag = []



for index,visitId in enumerate(listOfVisitId):

    #print(f"=========================={visitId}=============================")
    print(f"==================={index}) ====== {visitId} =============================")

    row_source = df_myselectedvisits.iloc[index]
    detector = row_source['detector']
    
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': detector}

    #retrieve the calexp
    calexp = butler.get('calexp', **dataId,collections=collections)

    # retrieve some information
    info_psf = calexp.getPsf()
    info_wcs = calexp.getWcs()
    pixelScale = info_wcs.getPixelScale().asArcseconds()
    info_photocalib = calexp.getPhotoCalib()
    
  

    # Select the subimage
   
    x_target = row_source['x']
    y_target = row_source['y']
    ra_target= row_source['ra']
    dec_target= row_source['dec']
    psfSigma = row_source['psfSigma']
    psfMagDiffMmag = row_source['psfMagDiffMmag'] 
    
    all_xytarget.append((x_target,y_target))

    print(f">>> detector {detector}, target ({x_target},{y_target}),......, psfSigma = {psfSigma}")

    xSrc = x_target
    ySrc = y_target
    targetPoint = geom.Point2D(xSrc,ySrc)
    
    minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
    maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
    srcBbox = geom.Box2I(minBbox, maxBbox)
    # Make the cutout

    # two ways to make cutout ExposureF or Factory
    subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
    
    # Generate the cutout image
    cutout = calexp.Factory(calexp, srcBbox, origin=afwImage.LOCAL, deep=False)
    #extent = (xmin,ymin,xmin+width,ymin+height)
    all_cutout.append(cutout)
    all_psfSigma.append(psfSigma) 
    all_psfMagDiffMmag.append(psfMagDiffMmag)


    ## psf shape
    psf_shape = info_psf.computeShape(targetPoint)
    print(">>>>>>> psf_shape :: ",psf_shape)

    ## psf sigma
    psf_sigma = psf_shape.getDeterminantRadius()
    psf_fwhm = psf_sigma * SIGMA_TO_FWHM
    print(f">>>>>.  psf_sigma = {psf_sigma:.3f} pixels  , psf_fwhm = {psf_fwhm:.3f} pixels ")


    ## psf : Aperture flux (normalized to 1)
    # Print the aperture flux within 1, 2, and 3$\sigma$.
    # Recall that the total flux of the PSF kernel has been normalized to an integrated flux of 1.
    psf_apflux_1s = info_psf.computeApertureFlux(radius=1.0*psf_sigma, position=targetPoint)
    psf_apflux_2s = info_psf.computeApertureFlux(radius=2.0*psf_sigma, position=targetPoint)
    psf_apflux_3s = info_psf.computeApertureFlux(radius=3.0*psf_sigma, position=targetPoint)
    print(f">>>>>> psf_apflux_1s = {psf_apflux_1s}, psf_apflux_2s = {psf_apflux_2s}, psf_apflux_3s = {psf_apflux_3s}")
        
    ## psf peak
    psf_peak = info_psf.computePeak(position=targetPoint)
    print(f">>>> psf_peak = {psf_peak}")   
 

    ## properties
    # The returned PSF properties are size (FWHM in pixels), 
    # aperture photometry flux (within a 1-sigma radius), 
    # peak flux value (recall that the PSF has been normalized to a sum of 1), 
    # and the dimensions of the PSF postage stamp (stamp display is demonstrated below).
    properties = getPsfProperties(info_psf, targetPoint)
    print(">>>> props = ", properties)


    ## Kernel
    # Use the extracted PSF information to create and display postage stamps of the PSF using
    # two functions: `computeKernelImage`, which uses central pixels `(0, 0)`,
    # and `computeImage`, which accepts user-specified central pixels.

    # Note that with `computeKernelImage`, the postage stamp's coordinates are centered at the origin 
    # of the image.
    # The coordinates of this origin point are (0, 0), 
    # resulting in negative coordinates for the lower left point.


    psf_calexp_kernel = info_psf.computeKernelImage(targetPoint)
    first_psf_image_calexp = psf_calexp_kernel.convertF()
    all_firstpsfimage.append(first_psf_image_calexp) 
    # The coordinates of this origin point are (0, 0), 
    # resulting in negative coordinates for the lower left point.
    print(">>> negative coordinates for the lower left point from computeKernelImage",first_psf_image_calexp.getXY0())                      

    # Use the `computeImage` method to retrieve the PSF kernel at the location of the `point_image` 
    # defined above as a postage stamp with a center pixel defined by `point_image`.
    # (I.e., instead of a PSF kernel postage stamp centered on `0, 0`, as returned by `computeKernelImage`).
    second_psf_image_calexp = info_psf.computeImage(targetPoint).convertF()
    all_secondpsfimage.append(second_psf_image_calexp) 

    
    
    #if index>=10:
    #    break

In [ ]:
NIMG = len(all_cutout)
NCOLS = 5
#NROWS = int(np.ceil(NIMG/NCOLS))
NROWS = NIMG

In [ ]:
fig, axes = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(4*NCOLS,4*NROWS))
for index,ax in enumerate(axes.flatten()):
    #if index//2 == NIMG:
    #    break

    # even number show the cutout
    if index%NCOLS  == 0:
        cutout = all_cutout[index//NCOLS].image.array
        visitId = listOfVisitId[index//NCOLS]
        #ax.imshow(cutout,interpolation="nearest")
        psfSigma = all_psfSigma[index//NCOLS]
        psfMagDiffMmag = all_psfMagDiffMmag[index//NCOLS]
        x_target,y_target = all_xytarget[index//NCOLS][0],all_xytarget[index//NCOLS][1]
        textstr = '\n'.join((
        r'$expos = %.0f$' % (visitId, ),
        r'$x_t = %.1f , y_t = %.1f$' % (x_target,y_target), 
        r'$psfMagDiff = %.0f mmag$' %(psfMagDiffMmag, ),  
        r'$\sigma_{PSF} = %.2f pix$' % (psfSigma , )))
        ann = Annulus((boxSize,boxSize),r=psfSigma,width=0.1,color="red")
        ax.imshow(cutout,aspect='equal')
        ax.add_patch(ann)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,
        verticalalignment='top',color="y", bbox=props)  
        title = f" {index//NCOLS}) {visitId}"
        ax.set_title(title)
    elif index%NCOLS  == 1:
        kernelimg = all_firstpsfimage[index//NCOLS].array
        ax.imshow(kernelimg, cmap="viridis",aspect='equal')
        ax.set_title("psf Kernel")
    elif index%NCOLS  == 2:
        #kernelimg2 = all_secondpsfimage[index//NCOLS].array
        #ax.imshow(kernelimg2, cmap="viridis")
        plotRadialAverage(all_firstpsfimage[index//NCOLS], ax=ax)
        ax.set_title("Radial profile")
        ax.grid()
    elif index%NCOLS == 3:
        plotContours(all_firstpsfimage[index//NCOLS], ax=ax)
        ax.set_title("Contour lines")
        ax.grid()
    elif index%NCOLS == 4:
        plotCurveOfGrowth(all_firstpsfimage[index//NCOLS], ax=ax)
        ax.set_title("Growth function")
        ax.grid()
    


plt.suptitle(suptitle,y=1.0)
plt.tight_layout()
plt.show()